In [1]:
import numpy as np
import plate_map as pm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from unidec_modules import unidectools as ud
from copy import deepcopy 
from unidec_modules.v3_SeqChrom import *
import scipy
from copy import deepcopy
import copy

In [2]:
folder = "D:\\2021 mbp time courses v2\\25"
map_path = "C:\\Users\\cm19ljc\\Documents\\GitHub\\UniDec\\reaction maps\\MBP labelling x5 25pc.xlsx"


In [3]:
filenames = os.listdir(folder)
engines = []
for dname, dirs, files in os.walk(folder):
    
    for fname in files:
        
        if fname[-4:] == "mzML":
            eng = SeqChrom()

            spectra_path = os.path.join(dname, fname)
            eng.load_mzml(spectra_path)
            print("loaded {}".format(spectra_path))
            engines.append(eng)


UniDec Engine v.5.0.1

UniDec Path: C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_bin\UniDec.exe

UniDec Engine v.5.0.1

UniDec Path: C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_bin\UniDec.exe
Opening HDF5 File: D:\2021 mbp time courses v2\25\12-10-21 E1_333416_RB7_01_79609.d.hdf5
Reading mzML: D:\2021 mbp time courses v2\25\12-10-21 E1_333416_RB7_01_79609.d.mzML
[Warning] Not index found and build_index_from_scratch is False
Converted to gzip file to improve speed: D:\2021 mbp time courses v2\25\12-10-21 E1_333416_RB7_01_79609.d.mzML.gz
Error getting TIC in mzML; trying to make it...


C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_modules\mzMLimporter.py:311: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.data = np.array(self.data)


Imported Data. Constructing TIC
Done
loaded D:\2021 mbp time courses v2\25\12-10-21 E1_333416_RB7_01_79609.d.mzML

UniDec Engine v.5.0.1

UniDec Path: C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_bin\UniDec.exe

UniDec Engine v.5.0.1

UniDec Path: C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_bin\UniDec.exe
Opening HDF5 File: D:\2021 mbp time courses v2\25\15-10-21 E1 25_333694_BE2_01_79676.d.hdf5
Reading mzML: D:\2021 mbp time courses v2\25\15-10-21 E1 25_333694_BE2_01_79676.d.mzML
[Warning] Not index found and build_index_from_scratch is False
Converted to gzip file to improve speed: D:\2021 mbp time courses v2\25\15-10-21 E1 25_333694_BE2_01_79676.d.mzML.gz
Error getting TIC in mzML; trying to make it...
Imported Data. Constructing TIC
Done
loaded D:\2021 mbp time courses v2\25\15-10-21 E1 25_333694_BE2_01_79676.d.mzML


In [4]:
def process_maps(self, groupby = 'Reaction'):

    

    # Reaction OR Substrate Conc

    self.pmap2 = self.pmap[self.pmap['Type'] != 'empty']
    self.pmap2.loc[:, 'Species'] = np.nan

    for skey, sval in self.speciesmap.groupby(['Reaction']):
        splist = [Species(spval.to_dict('records')[0], name = spkey) for spkey, spval in sval.groupby('Species')]
        splist = colorcodeclass(splist)

        for s in splist:
            self.pmap2.loc[:, s.__name__] = np.nan
            self.pmap2.loc[self.pmap2['Reaction']==skey, s.__name__] = self.pmap2.apply(lambda _:deepcopy(s), axis = 1)

        spnames = [s.__name__ for s in splist]
        self.pmap2.loc[self.pmap2['Reaction']==skey, 'Species'] = self.pmap2.apply(lambda _:spnames, axis = 1)

    # update_vars

    if len(self.data.spectra) == len(self.pmap2):
        for i, s in enumerate(self.data.spectra):
            well_id = self.pmap2.index[i]
            timevar = self.pmap2['Time'].iloc[i]
            s.attrs['Variable 1'] = well_id
            s.var1 = well_id
            s.attrs['Variable 2'] = timevar
            s.var2 = timevar
            self.pmap2.loc[well_id, 'Spectra'] = s



    spectra = {s.var1:s for s in self.data.spectra}

    # update species with well info/metadata 

    groupby = 'Reaction' # Reaction OR Substrate Conc

    for index, row in self.pmap2.iterrows():
        for specs in row['Species']:
            row[specs] = deepcopy(row[specs])
            row[specs].coord = row.name
    #         print(row[specs])
            vals = row[~row.index.isin(row['Species'])].to_dict()
            row[specs].__dict__.update(vals)
    return self
            

def peak_match(self, window = 10):
    window = 10

    intmat = np.array([])

    for index, row in self.pmap2.iterrows():

        rowints = np.array([])

        specieslist = list(row[row.index.isin(row['Species'])])
        theory_masses = np.array([sp.Mass for sp in specieslist])
        data_masses = np.array([p.mass for p in row['Spectra'].pks.peaks])
        pks = np.array([p for p in row['Spectra'].pks.peaks])

        # match algorithm 
        tm, dm = np.meshgrid(theory_masses, data_masses)
        diff = abs(tm - dm)
        diff[diff>window] = np.nan
        for i, d in enumerate(diff):
            if np.isnan(d).all()==False:
                minimum = np.nanargmin(d)
                data_peak = data_masses[i]

                specieslist[minimum].peak = pks[i]
                specieslist[minimum].integral = pks[i].integral[0]
                print("{}, {} = {}".format(row[row.index.isin(row['Species'])][minimum].__name__, data_peak, pks[i]))

                row[row.index.isin(row['Species'])][minimum].integral = pks[i].integral[0]
                row[row.index.isin(row['Species'])][minimum].peak = pks[i]
                np.append(rowints, pks[i].integral[0])
                print(row[row.index.isin(row['Species'])][minimum].integral)

    return self


In [5]:
def normalise_peaks(self):
    for index, row in self.pmap2.iterrows():
        ints = []
        for s in row[row['Species']]:
            if type(s.integral) != list:
                ints.append(s.integral)

    #     ints = np.array([s.integral for s in row[row['Species']]])
        sum_ints = np.sum(ints)
        for s in row[row['Species']]:
            if type(s.integral) != list:
                s.percentage = s.integral/sum_ints
            else:
                s.percentage = 0
            print("{}:{}".format(s.__name__, s.percentage))
            
    return self


In [6]:

plt.figure()
df1 = pd.DataFrame()
dflist = []
for eng in engines:
    try:
        eng.config.chrom_peak_width = 0.05 
        eng.get_chrom_peaks(lb = 1, ub = 5.5) 
        eng.add_chrom_peaks2() 
#         eng.plot_tic(peak_windows = True)
        eng.update_config(masslb = 35000, massub = 45000, minmz = 500, peakthresh = 0.1)
        eng.process_data()
        eng.run_unidec()
        eng.pick_peaks()
        eng.integrate_all()
#         eng.plot_all(dtype = 'massdat', combine = True, cmap = 'viridis', xlim = [41000, 43000])
        rmap = eng.upload_map(map_path)
        species, wells = eng.upload_map(map_path)
        eng = process_maps(eng)
        eng = peak_match(eng)
        eng = normalise_peaks(eng)
        self = eng
        # get data

        species = None
        datatype = 'percentage'
        rxndct = {}
        groupby = 'Reaction'

        # def extract_data(self):

        rxns_tc = []
        for k, v in self.pmap2.groupby(groupby):

            time = v['Time']
            speciesdct = {}
            speciestimedct = {}

            for index, row in v.iterrows():
                if species == None:
                    species = row.Species

                if len(species) == 1:
                    species = [species]

                for s in species:
                    if s in speciesdct:
                        speciesdct[s].append(getattr(row[s], datatype))
                        speciestimedct[s].append(row['Time'])

                    else:
                        speciesdct[s] = [getattr(row[s], datatype)]
                        speciestimedct[s] = [row['Time']]
            df = pd.DataFrame(speciesdct, index = time)
            rxns_tc.append(df)

        if len(rxns_tc) == 1:
            rxns_tc = rxns_tc[0]

        for name, y in speciesdct.items():
#             plt.figure()
            if name == "MBP-GVSEYG":
                plt.plot(time, y, label = name)
                plt.legend(loc = "center right")
                plt.title(eng.path)



        rxndct[k] = pd.DataFrame(speciesdct, index = time)
        rxndct[k].loc[:, 'Path'] = eng.path
        rxndct[k].loc[:, 'Reaction'] = k
        dflist.append(rxndct[k])
        # ---------------------------------------------------------------------------
        self.datadct = rxndct
    except Exception: 
        print("{} failed".format(eng.path))
plt.show()

1308 2
Noise Level: 0.12485162352255635 Window: 0.05
Bad Peak [0.41935    0.59168096] 0.24186700000000005 [0.41935, 0.661217]
Bad Peak [0.602567   0.50517104] 0.24603299999999995 [0.41935, 0.665383]
1.20155 0.13712999999999997
2.15345 0.13295000000000012
3.06365 0.1828299999999996
3.9904 0.18698000000000015
4.93377 0.17037999999999975
5.96038 0.06651999999999969
Getting scans: [136, 169]
Length merge axis: 38450
Getting scans: [366, 398]
Length merge axis: 37431
Getting scans: [586, 630]
Length merge axis: 38495
Getting scans: [811, 856]
Length merge axis: 38432
Getting scans: [1036, 1077]
Length merge axis: 39522
Execution Time: 0.1442418000000032
Execution Time: 6.7201357
Execution Time: 0.3106991000000008
Peak info saved to: D:\2021 mbp time courses v2\25\UniDec_Figures_and_Files\12-10-21 E1_333416_RB7_01_79609.d_extracts.txt
D:\2021 mbp time courses v2\25\12-10-21 E1_333416_RB7_01_79609.d.mzML failed
1307 2
Noise Level: 0.06088758097740972 Window: 0.05
Bad Peak [0.418733   0.655641

<Figure size 432x288 with 0 Axes>

In [7]:
data_df = pd.concat(dflist)

ValueError: No objects to concatenate

In [ ]:
key_species = "MBP-GVSEYG"
l = []
n = np.array([])
for name, df in data_df.groupby(['Path', 'Reaction']):
    
    df[key_species].plot(label = str(name[0][37:-7]))
    plt.legend(loc = 'upper right', bbox_to_anchor=(1.8, 1))
    l.append(np.array(df[key_species]))
arr = np.array(l)
print(arr)
yg = arr[:, 2] - arr[:, 0]
yg
t = data_df.index.unique()
xg = float(t[2]) -float(t[0])
g = yg/xg

# delete negative gradients 
arr = arr[g>0, :]
print(arr)

In [ ]:
std = np.std(arr, axis = 0)
mean = np.mean(arr, axis = 0)
# plt.plot(t, mean)
plt.errorbar(t, mean, std)
plt.xlabel('Time / s')
plt.ylabel('Normalised Area')
plt.show()